## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,51.04,83,89,16.33,overcast clouds
1,1,Hilo,US,19.7297,-155.0900,78.24,73,1,5.75,clear sky
2,2,Albany,US,42.6001,-73.9662,61.16,78,93,11.83,overcast clouds
3,3,Carnarvon,AU,-24.8667,113.6333,68.07,68,90,20.71,overcast clouds
4,4,Karlskrona,SE,56.1616,15.5866,50.45,95,98,9.37,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hilo,US,19.7297,-155.0900,78.24,73,1,5.75,clear sky
5,5,Pochutla,MX,15.7432,-96.4661,85.82,57,57,5.37,broken clouds
10,10,Axim,GH,4.8699,-2.2405,79.83,83,99,10.11,light rain
11,11,Gobabis,NaN,-22.4500,18.9667,80.29,6,0,9.89,clear sky
13,13,Grand-Santi,GF,4.2500,-54.3833,85.06,65,69,1.48,broken clouds
17,17,Rikitea,PF,-23.1203,-134.9692,74.16,87,93,13.40,moderate rain
18,18,Ancud,CL,-41.8697,-73.8203,71.13,41,24,4.05,few clouds
24,24,Cidreira,BR,-30.1811,-50.2056,71.71,79,17,21.79,few clouds
27,27,Saint-Philippe,RE,-21.3585,55.7679,72.99,83,28,7.65,scattered clouds
28,28,Savannah Bight,HN,16.4500,-85.8500,84.25,71,76,6.91,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                300
City                   300
Country                297
Lat                    300
Lng                    300
Max Temp               300
Humidity               300
Cloudiness             300
Wind Speed             300
Current Description    300
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how="any", axis=0)
preferred_cities_df.count()

City_ID                297
City                   297
Country                297
Lat                    297
Lng                    297
Max Temp               297
Humidity               297
Cloudiness             297
Wind Speed             297
Current Description    297
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,78.24,clear sky,19.7297,-155.0900,
5,Pochutla,MX,85.82,broken clouds,15.7432,-96.4661,
10,Axim,GH,79.83,light rain,4.8699,-2.2405,
13,Grand-Santi,GF,85.06,broken clouds,4.2500,-54.3833,
17,Rikitea,PF,74.16,moderate rain,-23.1203,-134.9692,
18,Ancud,CL,71.13,few clouds,-41.8697,-73.8203,
24,Cidreira,BR,71.71,few clouds,-30.1811,-50.2056,
27,Saint-Philippe,RE,72.99,scattered clouds,-21.3585,55.7679,
28,Savannah Bight,HN,84.25,broken clouds,16.4500,-85.8500,
31,Sakakah,SA,81.97,overcast clouds,29.9697,40.2064,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna(how="any", axis=0)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hilo,US,78.24,clear sky,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Pochutla,MX,85.82,broken clouds,15.7432,-96.4661,Hotel Posada San Jose
10,Axim,GH,79.83,light rain,4.8699,-2.2405,Axim Beach Resort And Spa
13,Grand-Santi,GF,85.06,broken clouds,4.2500,-54.3833,Hotel La Villa Couacou
17,Rikitea,PF,74.16,moderate rain,-23.1203,-134.9692,People ThankYou


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))